In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import glob

In [9]:
def box_from_baseline(baseline_points_str):
    baseline_points = np.array([p.split(",") for p in baseline_points_str.split(" ")], dtype=int)
    
    box_points = ""
    for i in range(len(baseline_points)):
        box_points += "{},{} ".format(baseline_points[i][0], baseline_points[i][1] + 7)
        
    for i in range(len(baseline_points)-1, -1, -1):
        box_points += "{},{} ".format(baseline_points[i][0], max(0, baseline_points[i][1] - 23))
    print(box_points)
    return box_points[:-1] #trim whitespace
        
box_from_baseline("10,64 174,48 418,38 531,32 710,37 838,34")

10,71 174,55 418,45 531,39 710,44 838,41 838,11 710,14 531,9 418,15 174,25 10,41 


'10,71 174,55 418,45 531,39 710,44 838,41 838,11 710,14 531,9 418,15 174,25 10,41'

In [10]:
# Load and parse the XML file

for filename in glob.glob("*.xml"):
    ns = {'ns': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}
    ET.register_namespace('', ns['ns'])

    tree = ET.parse(filename)
    root = tree.getroot()

    # Iterate through all TextRegion elements
    for text_region in root.findall('.//ns:TextRegion', ns):
        # Iterate through TextLine elements within each TextRegion
        for text_line in text_region.findall('.//ns:TextLine', ns):
            baseline = text_line.find('ns:Baseline', ns).get('points')
            updated_box_text = box_from_baseline(baseline)

            box = text_line.find('ns:Coords', ns)
            box.set('points', updated_box_text)
    print(tree)
    tree.write('modified/' + filename)

9,48 67,50 192,48 409,47 717,47 953,48 1101,46 1101,16 953,18 717,17 409,17 192,18 67,20 9,18 
29,83 149,81 302,83 513,83 735,83 904,83 1102,82 1102,52 904,53 735,53 513,53 302,53 149,51 29,53 
24,122 181,115 355,115 539,118 722,118 927,117 1058,116 1058,86 927,87 722,88 539,88 355,85 181,85 24,92 
34,154 240,150 474,150 665,152 856,151 1086,149 1086,119 856,121 665,122 474,120 240,120 34,124 
26,188 206,182 386,184 596,181 793,183 928,183 1094,181 1094,151 928,153 793,153 596,151 386,154 206,152 26,158 
29,220 208,218 385,213 454,217 605,211 783,214 1088,213 1088,183 783,184 605,181 454,187 385,183 208,188 29,190 
519,228 637,230 637,200 519,198 
29,252 232,246 496,248 688,246 928,247 1079,246 1079,216 928,217 688,216 496,218 232,216 29,222 
43,287 352,282 615,280 933,278 1093,277 1093,247 933,248 615,250 352,252 43,257 
27,318 173,313 341,312 534,314 704,309 795,311 1055,308 1055,278 795,281 704,279 534,284 341,282 173,283 27,288 
30,350 195,346 412,345 570,345 699,343 838,342 952,33

10,59 174,43 418,33 531,27 710,32 838,29 838,54 710,57 531,52 418,58 174,68 10,84 


'10,59 174,43 418,33 531,27 710,32 838,29 838,54 710,57 531,52 418,58 174,68 10,84 '